In [4]:
### https://pyfpdf.readthedocs.io/en/latest/Tutorial/index.html

from fpdf import FPDF
from src.mongodbclient import MongoDbClient
from src.cfgparser import getMongoDbStrConn
import os
from pathlib import Path
import pandas as pd

In [2]:
BASE_DIR = Path(os.path.abspath('')).resolve()

In [3]:
mongoClient = MongoDbClient(getMongoDbStrConn())

mongoRegBrCollection = mongoClient.getCollection('regbr','regbr_full_db')
mongoRegBrSentenceCollection = mongoClient.getCollection('regbr','regbr_sentences')

In [49]:
dfpredict_15082022_v1 = pd.read_excel(Path(os.path.join(BASE_DIR,'dataset','predicted','predict_15082022_v1.xlsx')))
dfpredict_15082022_v1 = dfpredict_15082022_v1[["FP","id_sentenca","id_doc_base"]].copy()
dfpredict_15082022_v1 = dfpredict_15082022_v1[dfpredict_15082022_v1["FP"]==0]


dfpredict_12082022_v1 = pd.read_excel(Path(os.path.join(BASE_DIR,'dataset','predicted','predict_12082022_v1.xlsx')))
dfpredict_12082022_v1 = dfpredict_12082022_v1[["FP","id_sentenca","id_doc_base"]].copy()
dfpredict_12082022_v1 = dfpredict_12082022_v1[dfpredict_12082022_v1["FP"]==0]


dfSentencas = pd.concat([dfpredict_15082022_v1,dfpredict_12082022_v1])

In [50]:
dfSentencas.drop_duplicates(inplace=True)
dfSentencas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 1 to 176
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   FP           107 non-null    int64 
 1   id_sentenca  107 non-null    object
 2   id_doc_base  107 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ KB


In [51]:
listaDocs = dfSentencas[["id_doc_base"]].drop_duplicates().id_doc_base.to_list()
listaIdSentencas = dfSentencas[["id_sentenca"]].drop_duplicates().id_sentenca.to_list()

In [52]:
### Create PDF



for idDocumento in listaDocs:
    
    pdf = FPDF('P', 'mm', 'A4')
    pdf.add_page()
    pdf.set_margins(5, 0, 5)
    pdf.set_text_color(0,0,0)
    pdf.set_font('Arial', '', 10)

    cursor = mongoRegBrCollection.find( {"id":idDocumento},{"id":1,"situacao":1,"tipo_lei":1,"agencia_reguladora":1,"ano":1,"situacao_infogov":1}) 

    for document in cursor:
        txt = f'id: {document["id"]}; situacao: {document["situacao"]}; tipo_lei: {document["tipo_lei"]}; '
        #pdf.multi_cell(0, 5, txt)
        
        pdf.cell(0, 5, f'id: {document["id"]}')
        pdf.ln()
        pdf.cell(0, 5, f'situacao: {document["situacao"]}')
        pdf.ln()
        pdf.cell(0, 5, f'tipo_lei: {document["tipo_lei"]}')
        pdf.ln()
        pdf.cell(0, 5, f'agencia_reguladora: {document["agencia_reguladora"]}')
        pdf.ln()
        pdf.cell(0, 5, f'ano: {document["ano"]}')
        pdf.ln()
        pdf.cell(0, 5, f'situacao_infogov: {document["situacao_infogov"]}')
        pdf.ln(10)
        # Adds a line beginning at point (10,30) and ending at point (110,30)
        pdf.line(5, 45, 205, 45)

    cursor = mongoRegBrSentenceCollection.find( {"id_doc_base":idDocumento},{"id_sentenca":1,"text_sentenca":1}) 

    for sentenca in cursor:
        
        if(sentenca['id_sentenca'] in  listaIdSentencas):
            pdf.set_text_color(220, 50, 50)
            pdf.set_font('Arial', 'B', 10)
        else:
            pdf.set_text_color(0,0,0)
            pdf.set_font('Arial', '', 10)
            
        pdf.multi_cell(w = 0, h = 10, txt=sentenca['text_sentenca'])
        pdf.ln()

    pathDoc = Path(os.path.join(BASE_DIR,'dataset','regbr_normas',f'{str(idDocumento)}.pdf'))
    pdf.output(pathDoc, 'F')